In [1]:
### #Loads the best_model after schnet training.
#user evaluates the model on the QM9 database

import schnetpack as spk
import torch
import schnetpack.nn 
import schnetpack.data
import pandas as pd
import scipy.linalg as la
from schnetpack.datasets import QM9

# Load best model where it was stored after training

model = torch.load("./trained_models/qm911/best_model", map_location=torch.device('cpu'))

# Download QM9 dataset to use in evaluating model
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

# Load split file 
train, val, test = spk.data.train_test_split(qm9data, split_file='./trained_models/qm911/split.npz')


device = 'cpu'

converter = spk.data.AtomsConverter(device=device)

#Control excel sheets to output data

from xlwt import Workbook

wb1 = Workbook()
wb2 = Workbook()
wb3 = Workbook()
wb4 = Workbook()
wb5 = Workbook()
wb6 = Workbook()
wb7 = Workbook()
wb8 = Workbook()
wb9 = Workbook()
wb10 = Workbook()
wb11 = Workbook()
wb12 = Workbook()
wb13 = Workbook()

sheet1 = wb1.add_sheet('Sheet1')
sheet2 = wb2.add_sheet('Sheet2')
sheet3 = wb3.add_sheet('Sheet3')
sheet4 = wb4.add_sheet('Sheet4')
sheet5 = wb5.add_sheet('Sheet5')
sheet6 = wb6.add_sheet('Sheet6')
sheet7 = wb7.add_sheet('Sheet7')
sheet8 = wb8.add_sheet('Sheet8')
sheet9 = wb9.add_sheet('Sheet9')
sheet10 = wb10.add_sheet('Sheet10')
sheet11 = wb11.add_sheet('Sheet10')
sheet12 = wb12.add_sheet('Sheet10')
sheet13 = wb13.add_sheet('Sheet10')

#Write the number of molecules you will evaluate (important for PCA)

#Some important counters for coding output into excel sheets 
index1=0
index2=0
index3=0
index4=0
index5=0
index6=0
index7=0
index8=0
index9=0
index10=0
index11=0
index12=0
index13=0

#BEGIN AUTOMATED CALCULATIONS, choose the range of QM9 to perform the calculations in
for idx in range(0,1):

# Load test set
    test_loader = spk.AtomsLoader(test, batch_size=100)
    converter = spk.data.AtomsConverter(device=device)
    
# Get properties of QM9
    at, props = qm9data.get_properties(idx)

    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()

    inputs = converter(at)
    print('Keys:', list(inputs.keys()))
    print('Truth:', props[QM9.U0].cpu().numpy()[0])

# Calculate prediction by best_model using schnetpack calculator 
    calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)
    at.set_calculator(calculator)

# Print energy prediction and index of molecule
    print('Prediction:', at.get_total_energy())
    print('idx',idx)

    
# Output atomization energies of each atom in the molecule
    import numpy as np

    from schnetpack.atomistic import output_modules

    atomenergies = (output_modules.yi)

    atomenergies = atomenergies.detach().numpy()
    atomenergies = atomenergies.astype(float)
    print(atomenergies[0][1])

    Z = props['_atomic_numbers']

    Z= Z.cpu().detach().numpy() 
     


# Write the xyz matrix for each molecule
    j = len(props['_positions'])
    print('number of atoms', j)
    for i in range(j):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 

# Extract atomenergies into a 1D array
    AE = np.zeros(len(z))
    for i in range(len(z)):
        AE[i] = atomenergies[0][i]
    print(AE)


# Import internal stuff from schnet to do PCA on

    from schnetpack.representation import SchNet
    activation = {}

    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    model = SchNet()
    model.embedding.register_forward_hook(get_activation('embedding'))
    output = model(inputs)
    print(activation['embedding'])
    


    from schnetpack.representation.schnet import x
    embed=x.clone()
    from schnetpack.representation.schnet import x0
    embed0=x0.clone()
    from schnetpack.representation.schnet import v
    from schnetpack.nn.cfconv import W
    filt=W.clone()
    from schnetpack.nn.activations import soft

    
# convert embed to a 2D array!
    emb = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            emb[i,j] = embed[0][i][j]
# convert embed0 to a 2D array!
    emb0 = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            emb0[i,j] = embed0[0][i][j]
#convert interaction layer to 2D array
    inter = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            inter[i,j] = v[0][i][j]
#convert soft to 2D array
    softact = np.zeros((len(z),15))
    for i in range(len(z)):
        for j in range(15):
            softact[i,j] = soft[0][i][j]
    
    
    
#Initial embedding vs final embedding after three interactions    
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(30):
                sheet1.write(index1,j,emb0[place,j])
            index1=index1+1
            break
    wb1.save('./pca/embedding0-H.xls')
    read_file = pd.read_excel (r'./pca/embedding0-H.xls')
    read_file.to_csv (r'./pca/embedding0-H.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 6:
            place=i
            for j in range(30):
                sheet2.write(index2,j,emb0[place,j])
            index2=index2+1
            break
    wb2.save('./pca/embedding0-C.xls')
    read_file = pd.read_excel (r'./pca/embedding0-C.xls')
    read_file.to_csv (r'./pca/embedding0-C.csv', index = None, header=True)

    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            for j in range(30):
                sheet3.write(index3,j,emb0[place,j])
            index3=index3+1
            break
    wb3.save('./pca/embedding0-N.xls')
    read_file = pd.read_excel (r'./pca/embedding0-N.xls')
    read_file.to_csv (r'./pca/embedding0-N.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(30):
                sheet4.write(index4,j,emb0[place,j])
            index4=index4+1
            break
    wb4.save('./pca/embedding0-O.xls')
    read_file = pd.read_excel (r'./pca/embedding0-O.xls')
    read_file.to_csv (r'./pca/embedding0-O.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(30):
                sheet5.write(index5,j,emb[place,j])
            index5=index5+1
            break
    wb5.save('./pca/embedding-O.xls')
    read_file = pd.read_excel (r'./pca/embedding-O.xls')
    read_file.to_csv (r'./pca/embedding-O.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(30):
                sheet6.write(index6,j,inter[place,j])
            index6=index6+1
            break
    wb6.save('./pca/interaction-O.xls')
    read_file = pd.read_excel (r'./pca/interaction-O.xls')
    read_file.to_csv (r'./pca/interaction-O.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(15):
                sheet7.write(index7,j,softact[place,j])
            index7=index7+1
            break
    wb7.save('./pca/softplus-O.xls')
    read_file = pd.read_excel (r'./pca/softplus-O.xls')
    read_file.to_csv (r'./pca/softplus-O.csv', index = None, header=True)
    
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(30):
                sheet8.write(index8,j,emb[place,j])
            index8=index8+1
            break
    wb8.save('./pca/embedding-H.xls')
    read_file = pd.read_excel (r'./pca/embedding-H.xls')
    read_file.to_csv (r'./pca/embedding-H.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(30):
                sheet9.write(index9,j,inter[place,j])
            index9=index9+1
            break
    wb9.save('./pca/interaction-H.xls')
    read_file = pd.read_excel (r'./pca/interaction-H.xls')
    read_file.to_csv (r'./pca/interaction-H.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(15):
                sheet10.write(index10,j,softact[place,j])
            index10=index10+1
            break
    wb10.save('./pca/softplus-H.xls')
    read_file = pd.read_excel (r'./pca/softplus-H.xls')
    read_file.to_csv (r'./pca/softplus-H.csv', index = None, header=True)

    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            for j in range(30):
                sheet11.write(index11,j,emb[place,j])
            index11=index11+1
            break
    wb11.save('./pca/embedding-N.xls')
    read_file = pd.read_excel (r'./pca/embedding-N.xls')
    read_file.to_csv (r'./pca/embedding-N.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            for j in range(30):
                sheet12.write(index12,j,inter[place,j])
            index12=index12+1
            break
    wb12.save('./pca/interaction-N.xls')
    read_file = pd.read_excel (r'./pca/interaction-N.xls')
    read_file.to_csv (r'./pca/interaction-N.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            for j in range(15):
                sheet13.write(index13,j,softact[place,j])
            index13=index13+1
            break
    wb13.save('./pca/softplus-N.xls')
    read_file = pd.read_excel (r'./pca/softplus-N.xls')
    read_file.to_csv (r'./pca/softplus-N.csv', index = None, header=True)



Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -1101.4878
Prediction: -1103.222900390625
idx 0
[-17.64036179]
number of atoms 5
C -2.8340169e-06 2.3049886e-06 -1.4378233e-07
H 0.014845718 -1.0918331 -0.0060250196
H 1.0244261 0.3779494 -0.007724565
H -0.52811974 0.36172476 -0.88464487
H -0.5111183 0.3521308 0.89839613
[-1032.66137695   -17.64036179   -17.64036179   -17.64036369
   -17.64036369]
tensor([[[-1.4384, -1.1577,  0.0607, -0.6069, -0.1070, -0.1345, -2.0492,
          -0.3532,  0.3640,  0.5868, -0.8481, -0.6312,  0.7976, -1.8633,
          -0.9906,  0.4684, -2.2403,  0.4618,  0.4116,  0.8957, -0.7625,
           0.8394,  2.6107, -0.7445,  0.1949, -0.1339, -0.9558,  2.2043,
          -0.1760,  1.9354, -0.9818,  2.1851,  0.6842,  1.7466,  0.8193,
           0.5308,  0.6836,  0.0763,  0.3281, -1.0114, -0.0483,  2.0633,
           0.7288, -0.4130, -0.8616, -0.7027, -0.4109,  2.9222, -0.7925,
          -0.3921, -0

/home/amerelsamman/Programs/anaconda3/lib/python3.7/site-packages/schnetpack/data/atoms.py:330: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  properties[pname] = torch.FloatTensor(row.data[pname])


FileNotFoundError: [Errno 2] No such file or directory: './pca/embedding0-H.xls'

In [ ]:


    from schnetpack.representation.schnet import x
    embed=x.clone()
    from schnetpack.representation.schnet import x0
    embed0=x0.clone()
    from schnetpack.representation.schnet import v
    from schnetpack.nn.cfconv import W
    filt=W.clone()
    from schnetpack.nn.activations import soft

    
# convert embed to a 2D array!
    emb = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            emb[i,j] = embed[0][i][j]
# convert embed0 to a 2D array!
    emb0 = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            emb0[i,j] = embed0[0][i][j]
#convert interaction layer to 2D array
    inter = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            inter[i,j] = v[0][i][j]
#convert soft to 2D array
    softact = np.zeros((len(z),15))
    for i in range(len(z)):
        for j in range(15):
            softact[i,j] = soft[0][i][j]
    
    

#Initial embedding vs final embedding after three interactions    
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(30):
                sheet1.write(index1,j,emb0[place,j])
            index1=index1+1
            break
    wb1.save('./pca/embedding0-H.xls')
    read_file = pd.read_excel (r'./pca/embedding0-H.xls')
    read_file.to_csv (r'./pca/embedding0-H.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 6:
            place=i
            for j in range(30):
                sheet2.write(index2,j,emb0[place,j])
            index2=index2+1
            break
    wb2.save('./pca/embedding0-C.xls')
    read_file = pd.read_excel (r'./pca/embedding0-C.xls')
    read_file.to_csv (r'./pca/embedding0-C.csv', index = None, header=True)

    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            for j in range(30):
                sheet3.write(index3,j,emb0[place,j])
            index3=index3+1
            break
    wb3.save('./pca/embedding0-N.xls')
    read_file = pd.read_excel (r'./pca/embedding0-N.xls')
    read_file.to_csv (r'./pca/embedding0-N.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(30):
                sheet4.write(index4,j,emb0[place,j])
            index4=index4+1
            break
    wb4.save('./pca/embedding0-O.xls')
    read_file = pd.read_excel (r'./pca/embedding0-O.xls')
    read_file.to_csv (r'./pca/embedding0-O.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(30):
                sheet5.write(index5,j,emb[place,j])
            index5=index5+1
            break
    wb5.save('./pca/embedding-O.xls')
    read_file = pd.read_excel (r'./pca/embedding-O.xls')
    read_file.to_csv (r'./pca/embedding-O.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(30):
                sheet6.write(index6,j,inter[place,j])
            index6=index6+1
            break
    wb6.save('./pca/interaction-O.xls')
    read_file = pd.read_excel (r'./pca/interaction-O.xls')
    read_file.to_csv (r'./pca/interaction-O.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(15):
                sheet7.write(index7,j,softact[place,j])
            index7=index7+1
            break
    wb7.save('./pca/softplus-O.xls')
    read_file = pd.read_excel (r'./pca/softplus-O.xls')
    read_file.to_csv (r'./pca/softplus-O.csv', index = None, header=True)
    
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(30):
                sheet8.write(index8,j,emb[place,j])
            index8=index8+1
            break
    wb8.save('./pca/embedding-H.xls')
    read_file = pd.read_excel (r'./pca/embedding-H.xls')
    read_file.to_csv (r'./pca/embedding-H.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(30):
                sheet9.write(index9,j,inter[place,j])
            index9=index9+1
            break
    wb9.save('./pca/interaction-H.xls')
    read_file = pd.read_excel (r'./pca/interaction-H.xls')
    read_file.to_csv (r'./pca/interaction-H.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(15):
                sheet10.write(index10,j,softact[place,j])
            index10=index10+1
            break
    wb10.save('./pca/softplus-H.xls')
    read_file = pd.read_excel (r'./pca/softplus-H.xls')
    read_file.to_csv (r'./pca/softplus-H.csv', index = None, header=True)

    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            for j in range(30):
                sheet11.write(index11,j,emb[place,j])
            index11=index11+1
            break
    wb11.save('./pca/embedding-N.xls')
    read_file = pd.read_excel (r'./pca/embedding-N.xls')
    read_file.to_csv (r'./pca/embedding-N.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            for j in range(30):
                sheet12.write(index12,j,inter[place,j])
            index12=index12+1
            break
    wb12.save('./pca/interaction-N.xls')
    read_file = pd.read_excel (r'./pca/interaction-N.xls')
    read_file.to_csv (r'./pca/interaction-N.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            for j in range(15):
                sheet13.write(index13,j,softact[place,j])
            index13=index13+1
            break
    wb13.save('./pca/softplus-N.xls')
    read_file = pd.read_excel (r'./pca/softplus-N.xls')
    read_file.to_csv (r'./pca/softplus-N.csv', index = None, header=True)


# Write the target properties to an excel file for PCA analysis